In [1]:
import numpy
import torch
import mobster as mb

import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
data = mb.load_example_data("./")


In [6]:
import pyro

pyro.enable_validation(True)
fitted = mb.fit_mobster(data,3,1, subclonal_prior="Moyal", multi_tail = False,
                        purity = 0.89,  truncated_pareto=True, max_it = 2000,
                        lr = 0.01,  e = 1e-10, lrd_gamma = 0.1,k_means_init = False,
        number_of_trials_subclonal= 1500, prior_lims_k=[1.,10000.],
                        prior_lims_clonal=[1,10000.],
                        number_of_trials_clonal_mean= 300, CUDA = False)


Running MOBSTER on 3 karyotypes with 3 subclones.
Fitting a model with tail


ELBO: 8029.278610035  : 100%|██████████| 2000/2000 [01:28<00:00, 22.65it/s]

Computing cluster assignements.


Computing information criteria.
Done!



In [5]:



mb.plot_results(data, fitted, output= "cane.png", drivers=None)